In [0]:
## FUNÇÕES UTILIZADAS => (GROUP BY) / (ORDER BY = SORT ) / (date_format) / (RPAD + LPAD) / (regexp_replace) / (SUBSTRING)

In [0]:
# IMPORTANDO BIBLIOTECAS PARA TRABALHAR COM (SPARK)
import pyspark.sql.functions 
from pyspark.sql.types import StructType, StructField , StringType
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
# CRIANDO SESSÃO DO SPARK 

spark = (SparkSession.builder
        .master("local(*)")
        .appName("Estudos-Pyspark-1")
        .getOrCreate())

In [0]:
spark # informações sobre o spark 

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# CRIANDO DATAFRAME ==> DADOS ==> TUPLA CONTENDO VARIAS LISTAS 

dados = (
    ['1','José','Anápolis','São Paulo','01-09-1900'],
    ['2','Igor','Anápolis','São Paulo','11-09-1977'],
    ['3','Leonardo','Anápolis','São Paulo','21-12-2000'],
    ['4','Humberto','Pato Branco','Rio Grande do Sul','13-11-1964'],
    ['5','Isaías','Pato Branco','Rio Grande do Sul','07-07-2002'],
    ['6','Lucas','Tauá','Ceará','05-09-1984']
    
)

In [0]:
# CRIANDO COLUNAS SHEMA DE DADOS ==> STRUCT TYPE 

colunas = (StructType([
    StructField('cod_cliente' , StringType(), True),
    StructField('nome' , StringType(), True),
    StructField('municipio' , StringType(), True),
    StructField('estado' , StringType(), True),
    StructField('data_nasc' , StringType(), True),
]))

In [0]:
# CRIANDO DATAFRAME FINAL 

df = spark.createDataFrame(data = dados , schema = colunas)

In [0]:
# VISUALIZANDO DATA FRAME 
df.display()

cod_cliente,nome,municipio,estado,data_nasc
1,José,Anápolis,São Paulo,01-09-1900
2,Igor,Anápolis,São Paulo,11-09-1977
3,Leonardo,Anápolis,São Paulo,21-12-2000
4,Humberto,Pato Branco,Rio Grande do Sul,13-11-1964
5,Isaías,Pato Branco,Rio Grande do Sul,07-07-2002
6,Lucas,Tauá,Ceará,05-09-1984


In [0]:
# 1) EXIBA (A QUANTIDADE) DE MUNICIPIOS E ORDENE EM ORDEM CRESCENTE POR MUNICIPIO => (GROUP BY + COUNT)

df.groupBy('municipio').count().sort('municipio', ascending=True).show()


+-----------+-----+
|  municipio|count|
+-----------+-----+
|   Anápolis|    3|
|Pato Branco|    2|
|       Tauá|    1|
+-----------+-----+



In [0]:
# 2) SELECIONE OS (NOMES) POR (ORDEM CRESCENTE) DO MENOR PARA O MAIOR ==> (ORDER BY OU SORT) => por PADRAO JA VEM COMO ASC...

df.orderBy('nome').show() # --> TAMBEM DA ->  df.sort('nome').show() 


+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|
|          1|    José|   Anápolis|        São Paulo|01-09-1900|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|
+-----------+--------+-----------+-----------------+----------+



In [0]:
# 3) ORDENE OS (ESTADOS) DO MAIOR PARA O MENOR (DESCRECENTE) => ORDEM ALFABÉTICA  (ORDER BY OU SORT)

df.select('estado').orderBy(col('estado').desc()).show()

#df.select('estado').sort(col('estado').desc()).show() # MESMA COISA 

+-----------------+
|           estado|
+-----------------+
|        São Paulo|
|        São Paulo|
|        São Paulo|
|Rio Grande do Sul|
|Rio Grande do Sul|
|            Ceará|
+-----------------+



In [0]:
# 4) CRIE UMA NOVA COLUNA PARA (IDADE), PEGANDO A DATA ATUAL - O (ANO) DA COLUNA (DATA_NAS)

data = df.withColumn('idade', date_format(current_timestamp(), 'yyyy').cast('integer') - substring(col('data_nasc'), 7,4).cast('integer'))

data.show()

+-----------+--------+-----------+-----------------+----------+-----+
|cod_cliente|    nome|  municipio|           estado| data_nasc|idade|
+-----------+--------+-----------+-----------------+----------+-----+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|  122|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|   45|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|   22|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|   58|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|   20|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|   38|
+-----------+--------+-----------+-----------------+----------+-----+



In [0]:
# 5) NO CAMPO (COD_CLIENTE) ==> COLOCAR 3 ZEROS A ESQUERDA DO COD
dados = df.withColumn('cod_cliente' , lpad(df.cod_cliente, 4, '0'))

dados.show()

+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|       0001|    José|   Anápolis|        São Paulo|01-09-1900|
|       0002|    Igor|   Anápolis|        São Paulo|11-09-1977|
|       0003|Leonardo|   Anápolis|        São Paulo|21-12-2000|
|       0004|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|       0005|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|
|       0006|   Lucas|       Tauá|            Ceará|05-09-1984|
+-----------+--------+-----------+-----------------+----------+



In [0]:
# 6) adicionar 2 (x) a direita do cod_cliente 

test = df.withColumn('cod_cliente', rpad(df.cod_cliente, 3, 'x'))

test.show()

+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|        1xx|    José|   Anápolis|        São Paulo|01-09-1900|
|        2xx|    Igor|   Anápolis|        São Paulo|11-09-1977|
|        3xx|Leonardo|   Anápolis|        São Paulo|21-12-2000|
|        4xx|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|        5xx|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|
|        6xx|   Lucas|       Tauá|            Ceará|05-09-1984|
+-----------+--------+-----------+-----------------+----------+



In [0]:
# 7) CRIE UMA COLUNA COM A (DATA E HORA ATUAL)

dados = df.withColumn('dt/hora_atual', date_format(current_timestamp(), 'dd-MM-yyyy HH:mm:ss'))

dados.show()

+-----------+--------+-----------+-----------------+----------+-------------------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|      dt/hora_atual|
+-----------+--------+-----------+-----------------+----------+-------------------+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|13-08-2022 20:06:52|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|13-08-2022 20:06:52|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|13-08-2022 20:06:52|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|13-08-2022 20:06:52|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|13-08-2022 20:06:52|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|13-08-2022 20:06:52|
+-----------+--------+-----------+-----------------+----------+-------------------+



In [0]:
# 8) CRIE UMA COLUNA COM A (DATA ATUAL)

dados = df.withColumn('data_atual' , date_format(current_timestamp(), 'dd-MM-yyyy'))

dados.show()

+-----------+--------+-----------+-----------------+----------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|data_atual|
+-----------+--------+-----------+-----------------+----------+----------+
|          1|    José|   Anápolis|        São Paulo|01-09-1900|13-08-2022|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|13-08-2022|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|13-08-2022|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|13-08-2022|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|13-08-2022|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|13-08-2022|
+-----------+--------+-----------+-----------------+----------+----------+



In [0]:

# 9 ) tirar todos os (ACENTOS) DA COLUNA (ESTADO)

#dados = df.withColumn('estado', regexp_replace('estado', 'á' , 'a')).withColumn('estado', regexp_replace('estado', 'ã' , 'a'))

dados = df.withColumn('estado', regexp_replace('estado', 'á' , 'a')).withColumn('estado', regexp_replace('estado', 'ã' , 'a'))

dados.show()

+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|          1|    José|   Anápolis|        Sao Paulo|01-09-1900|
|          2|    Igor|   Anápolis|        Sao Paulo|11-09-1977|
|          3|Leonardo|   Anápolis|        Sao Paulo|21-12-2000|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|
|          6|   Lucas|       Tauá|            Ceara|05-09-1984|
+-----------+--------+-----------+-----------------+----------+



In [0]:
# 10) REMOVER DA COLUNA NOME OS (ACENTOS) TAMBEM 

dados = df.withColumn('nome' , regexp_replace('nome', 'é' , 'e')).withColumn('nome', regexp_replace('nome', 'í' , 'i'))

dados.show()

+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|          1|    Jose|   Anápolis|        São Paulo|01-09-1900|
|          2|    Igor|   Anápolis|        São Paulo|11-09-1977|
|          3|Leonardo|   Anápolis|        São Paulo|21-12-2000|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|          5|  Isaias|Pato Branco|Rio Grande do Sul|07-07-2002|
|          6|   Lucas|       Tauá|            Ceará|05-09-1984|
+-----------+--------+-----------+-----------------+----------+



In [0]:
# 11) REMOVER DA COLUNA (MUNICIPIO) OS ACENTOS 

dados = df.withColumn('municipio' , regexp_replace('municipio' , 'á' , 'a' ))

dados.show()

+-----------+--------+-----------+-----------------+----------+
|cod_cliente|    nome|  municipio|           estado| data_nasc|
+-----------+--------+-----------+-----------------+----------+
|          1|    José|   Anapolis|        São Paulo|01-09-1900|
|          2|    Igor|   Anapolis|        São Paulo|11-09-1977|
|          3|Leonardo|   Anapolis|        São Paulo|21-12-2000|
|          4|Humberto|Pato Branco|Rio Grande do Sul|13-11-1964|
|          5|  Isaías|Pato Branco|Rio Grande do Sul|07-07-2002|
|          6|   Lucas|       Taua|            Ceará|05-09-1984|
+-----------+--------+-----------+-----------------+----------+

